# セッション検出

このノートブックは、オフライン評価のために AgentCore Observability からエージェントセッションを検出します。エージェントのトレースロググループにクエリを実行してセッションを見つけ、分析ノートブックで処理するために JSON ファイルに保存します。

**2つの検出方法:**

1. **時間ベース**: 時間枠内のすべてのセッションを検出。最近のエージェントアクティビティの一括評価に使用します。

2. **スコアベース**: AgentCore からの既存の評価スコアでセッションを検出。更新されたルーブリックで低スコアのセッションを再評価するために使用します。

**出力:** 分析ノートブック用のセッション ID とメタデータを含む `discovered_sessions.json`。

## この位置づけ

これは評価ワークフローの**ノートブック1**です。ここでセッションを検出した後、2つの評価パスのいずれかを選択します。

![Notebook Workflow](images/notebook_workflow.svg)

## セットアップ

必要なモジュールをインポートし、`config.py` から設定を読み込みます。すべての設定値は、このセルを実行する前に環境変数で上書きできます。

In [ ]:
import logging
import sys
from datetime import datetime, timedelta, timezone

sys.path.insert(0, ".")

from config import (
    AWS_REGION,
    SOURCE_LOG_GROUP,
    EVAL_RESULTS_LOG_GROUP_FULL,
    LOOKBACK_HOURS,
    MAX_SESSIONS,
    MIN_SCORE,
    MAX_SCORE,
    DISCOVERED_SESSIONS_PATH,
    EVALUATOR_NAME,
)

from utils import (
    ObservabilityClient,
    SessionDiscoveryResult,
    # SessionInfo,
)

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
)
logger = logging.getLogger(__name__)

## 設定

スコアベース検出に使用される `EVALUATOR_NAME` は `config.py` から読み込まれます。これは既存の評価結果の評価メトリクス名と一致する必要があります。`LOOKBACK_HOURS`、`MAX_SESSIONS`、スコアしきい値などの設定を変更するには `config.py` を修正してください。

In [ ]:
# EVALUATOR_NAME is loaded from config.py
print(f"Using evaluator: {EVALUATOR_NAME}")

## クライアントの初期化

CloudWatch Logs Insights クエリを処理する `ObservabilityClient` を作成します。時間範囲は config の `LOOKBACK_HOURS` から計算されます。

In [ ]:
obs_client = ObservabilityClient(
    region_name=AWS_REGION,
    log_group=SOURCE_LOG_GROUP,
)

end_time = datetime.now(timezone.utc)
start_time = end_time - timedelta(hours=LOOKBACK_HOURS)
start_time_ms = int(start_time.timestamp() * 1000)
end_time_ms = int(end_time.timestamp() * 1000)

## 時間ベースの検出

AgentCore Observability ロググループに対して、時間枠内のすべての一意のセッション ID をクエリします。最新のアクティビティでソートされたスパン数とタイムスタンプを含むセッションを返します。最近のすべてのエージェントインタラクションを評価したい場合にこの方法を使用します。

In [ ]:
time_based_sessions = obs_client.discover_sessions(
    start_time_ms=start_time_ms,
    end_time_ms=end_time_ms,
    limit=MAX_SESSIONS,
)

print(f"Discovered {len(time_based_sessions)} sessions")

## スコアベースの検出

AgentCore 評価結果ロググループにクエリを実行して、既存の評価スコアでセッションを検索します。指定した評価メトリクスが `MIN_SCORE` と `MAX_SCORE` の間でスコア付けしたセッションをフィルタリングします。更新されたルーブリックで再評価するためにパフォーマンスの低いセッションを見つけるにはこの方法を使用します。

In [ ]:
score_based_sessions = obs_client.discover_sessions_by_score(
    evaluation_log_group=EVAL_RESULTS_LOG_GROUP_FULL,
    evaluator_name=EVALUATOR_NAME,
    start_time_ms=start_time_ms,
    end_time_ms=end_time_ms,
    min_score=MIN_SCORE,
    max_score=MAX_SCORE,
    limit=MAX_SESSIONS,
)

print(f"Discovered {len(score_based_sessions)} sessions by score")

## 検出方法の選択

使用する検出セッションのセットを選択します。時間ベースの結果には `USE_TIME_BASED = True` を設定し、スコアベースの結果には `False` を設定します。選択されたセッションは、検出方法に関するメタデータとともに `SessionDiscoveryResult` にパッケージ化されます。

In [ ]:
# Set to False to use score-based discovery instead
USE_TIME_BASED = True

if USE_TIME_BASED:
    selected_sessions = time_based_sessions
    discovery_method = "time_based"
    log_group = SOURCE_LOG_GROUP
    filter_criteria = None
else:
    selected_sessions = score_based_sessions
    discovery_method = "score_based"
    log_group = EVAL_RESULTS_LOG_GROUP_FULL
    filter_criteria = {
        "evaluator_name": EVALUATOR_NAME,
        "min_score": MIN_SCORE,
        "max_score": MAX_SCORE,
    }

discovery_result = SessionDiscoveryResult(
    sessions=selected_sessions,
    discovery_time=datetime.now(timezone.utc),
    log_group=log_group,
    time_range_start=start_time,
    time_range_end=end_time,
    discovery_method=discovery_method,
    filter_criteria=filter_criteria,
)

print(f"Selected {len(selected_sessions)} sessions via {discovery_method}")

## セッションのプレビュー

検出された最初の10セッションを表示します。時間ベースの検出では、セッション ID とスパン数を表示します。スコアベースの検出では、セッション ID と平均評価スコアを表示します。

In [ ]:
for i, session in enumerate(selected_sessions[:10]):
    meta = session.metadata or {}
    if discovery_method == "time_based":
        print(f"{i+1}. {session.session_id} - {session.span_count} spans")
    else:
        print(f"{i+1}. {session.session_id} - avg_score: {meta.get('avg_score', 0):.2f}")

## 結果の保存

検出結果を JSON に保存します。このファイルは分析ノートブックによって読み込まれ、各セッションを処理します。出力パスは `config.py` で `DISCOVERED_SESSIONS_PATH` として設定されています。

In [ ]:
discovery_result.save_to_json(DISCOVERED_SESSIONS_PATH)
print(f"Saved {len(selected_sessions)} sessions to {DISCOVERED_SESSIONS_PATH}")

## 出力の検証

JSON ファイルが正しく保存されたことを確認します。検証後、マルチセッション分析ノートブックに進んでこれらのセッションを評価します。

In [ ]:
import json

with open(DISCOVERED_SESSIONS_PATH, "r") as f:
    saved_data = json.load(f)

print(f"Sessions: {len(saved_data['sessions'])}")
print(f"Method: {saved_data['discovery_method']}")
print(f"Time range: {saved_data['time_range_start']} to {saved_data['time_range_end']}")